In [1]:
import sys
import os 
import numpy as np
import time
import tensorflow as tf
from PIL import Image
from tensorflow.python.framework import graph_util  
from tensorflow.python.platform import gfile  
#myself
import src.make_siameseFC as siam
from src.train_siam_net import train_siam_net
from src.region_to_bbox import region_to_bbox
from src.parse_arguments import parse_arguments

_conv_w_sz=np.array([11,5,3,3,3]) #the map size of filter(weight of conv net)
_conv_w_in_c=np.array([3,96,256,384,384])# the input channle number of filter
_conv_w_out=np.array([96,256,384,384,256])# the output number of feature map(the number of filter kernel)

def main():
    #avoid printing TF debugging information(only show error log)
    os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
    hp,evaluation,run,env,design=parse_arguments()
    #build TF graph in siamese once for all
    #siam.init_create_net()
    filename,siam_net_z,loss,train_op=siam.make_siameseFC(env,design,hp)
    
    #iterate through all videos of evaluation.dataset
    if evaluation.video == 'all':
        #the path of folder of all videos
        train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset)
        videos_list=[v for v in os.listdir(train_data_folder)]
        videos_list.sort()
        num_v=len(videos_list)
        for i in range(num_v):
            gt,frame_name_list,frame_sz,n_frames=_init_train_video(env,evaluation,videos_list[i])
            start_frame=evaluation.start_frame
            #not sure
            #gt_=gt[start_frame:,:]
            gt_=gt[start_frame:]
            frame_name_list_=frame_name_list[start_frame:]
            num_frames=np.size(frame_name_list_)
            
            for j in range(num_frames-1):
                pos_x,pos_y,target_w,target_h=region_to_bbox(gt_[j])
                #train_siam_net(design,hp,frame_name_list,z_index,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss)
                train_siam_net(design,hp,frame_name_list,j,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss,train_op)
        
    else:
        gt,frame_name_list,_,_ = _init_train_video(env,evaluation,evaluation.video)
        start_frame=evaluation.start_frame
        gt_=gt[start_frame:]
        frame_name_list_=frame_name_list[start_frame:]
        num_frames=np.size(frame_name_list_)
        for i in range(num_frames-1):
            pos_x,pos_y,target_w,target_h=region_to_bbox(gt[evaluation.start_frame])
            train_siam_net(design,hp,frame_name_list,i,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss,train_op)
            
    write_file_param(design,env,evaluation)

In [2]:
def _init_train_video(env,evaluation,video):
     #-------------------------------------------------------------------------
    #function//init info of a train video sequence
    #-------------------------------------------------------------------------
    #the path of train_data folder
    train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset,video)
    #os.listdir():show the file list of the folder
    frame_name_list=[f for f in os.listdir(train_data_folder) if f.endswith("."+env.image_type)]
    frame_name_list=[os.path.join(train_data_folder,'')+s for s in frame_name_list]
    frame_name_list.sort()
    
    #get the info of first frame
    with Image.open(frame_name_list[0]) as img:
        frame_sz=np.asarray(img.size)
        ##????????????????????????????????????????????
        frame_sz[1],frame_sz[0]=frame_sz[0],frame_sz[1]
        
        #read the initialization from ground truth(init the template_z)
    gt_file=os.path.join(train_data_folder,evaluation.gt_name)
    gt=np.genfromtxt(gt_file,delimiter=evaluation.gt_delimiter)
    num_frame=len(frame_name_list)
    assert num_frame == len(gt),'number of frame and number of gt should be the same'
    
    return gt,frame_name_list,frame_sz,num_frame

In [3]:
'''def write_file_param(design,env,evaluation):
    _layers_num=design.layers_num
    
    #get the time stamp now
    time_s=time.strftime('%Y-%m-%d-%H:%M:%S',time.localtime(time.time()))
    re_filename=str(time_s)+evaluation.train_re_param_file
    re_file_path=os.path.join(env.root_train_result_param,re_filename)
    for i in range(_layers_num):
        scope_name='conv'+str(i+1)
        with tf.variable_scope(scope_name,reuse=True):
            W_=tf.get_variable("W",shape=[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]])
            b_=tf.get_variable("b",shape=[_conv_w_out[i]])
            w_name='W'+str(i)
            b_name='b'+str(i)
            W_=tf.get_variable(w_name,shape=[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]],tf.constant_initalizer(W_))
            b_=tf.get_variable(b_name,shape=[_conv_w_out[i]],tf.constant_initalizer(b_))
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(_layers_num):
            w_name='W'+str(i)
            b_name='b'+str(i)
            graph_w=graph_util.convert_variables_to_constants(sess,sess.graph_def,[w_name])
            tf.train.wirte_graph(graph_w,'.',re_file_path,as_text=False)
            graph_b=graph_util.convert_variables_to_constants(sess,sess.graph_def,[b_name])
            tf.train.write_graph(graph_b,'.',re_file_path,as_text=False)'''



'def write_file_param(design,env,evaluation):\n    _layers_num=design.layers_num\n    \n    #get the time stamp now\n    time_s=time.strftime(\'%Y-%m-%d-%H:%M:%S\',time.localtime(time.time()))\n    re_filename=str(time_s)+evaluation.train_re_param_file\n    re_file_path=os.path.join(env.root_train_result_param,re_filename)\n    for i in range(_layers_num):\n        scope_name=\'conv\'+str(i+1)\n        with tf.variable_scope(scope_name,reuse=True):\n            W_=tf.get_variable("W",shape=[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]])\n            b_=tf.get_variable("b",shape=[_conv_w_out[i]])\n            w_name=\'W\'+str(i)\n            b_name=\'b\'+str(i)\n            W_=tf.get_variable(w_name,shape=[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]],tf.constant_initalizer(W_))\n            b_=tf.get_variable(b_name,shape=[_conv_w_out[i]],tf.constant_initalizer(b_))\n    with tf.Session() as sess:\n        sess.run(tf.global_variables_initializer())\n       

In [4]:
def write_file_param(design,env,evaluation):
    _layers_num=design.layers_num
    
    #get the time stamp now
    time_s=time.strftime('%Y-%m-%d-%H:%M:%S',time.localtime(time.time()))
    re_filename=str(time_s)+evaluation.train_re_param_file
    re_file_path=os.path.join(env.root_train_result_param,re_filename)
    for i in range(_layers_num):
        scope_name='conv'+str(i+1)
        print(scope_name)
        with tf.variable_scope(scope_name,reuse=True):
            #W_=tf.get_variable("W",shape=[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]])
            #b_=tf.get_variable("b",shape=[_conv_w_out[i]])
            W_=tf.get_variable("W",[_conv_w_sz[i],_conv_w_sz[i],_conv_w_in_c[i],_conv_w_out[i]])
            b_=tf.get_variable("b",[_conv_w_out[i]])
            with tf.Session() as sess:
                sess.run(tf.Print(W_,[W_]))
                sess.run(tf.Print(b_,[b_]))
          

In [5]:
if __name__=='__main__':
    sys.exit(main())

Layer conv1
Tensor("conv1/Relu:0", shape=(1, 123, 123, 96), dtype=float32)
Tensor("conv1/Relu_1:0", shape=(1, 59, 59, 96), dtype=float32)
_pool_stride
Tensor("conv1/pool1:0", shape=(1, 61, 61, 96), dtype=float32)
Tensor("conv1/pool1_1:0", shape=(1, 29, 29, 96), dtype=float32)
Layer conv2
Tensor("conv2/Relu:0", shape=(1, 57, 57, 256), dtype=float32)
Tensor("conv2/Relu_1:0", shape=(1, 25, 25, 256), dtype=float32)
_pool_stride
Tensor("conv2/pool2:0", shape=(1, 28, 28, 256), dtype=float32)
Tensor("conv2/pool2_1:0", shape=(1, 12, 12, 256), dtype=float32)
Layer conv3
Tensor("conv3/Relu:0", shape=(1, 26, 26, 384), dtype=float32)
Tensor("conv3/Relu_1:0", shape=(1, 10, 10, 384), dtype=float32)
Tensor("conv3/Relu:0", shape=(1, 26, 26, 384), dtype=float32)
Tensor("conv3/Relu_1:0", shape=(1, 10, 10, 384), dtype=float32)
Layer conv4
Tensor("conv4/Relu:0", shape=(1, 24, 24, 384), dtype=float32)
Tensor("conv4/Relu_1:0", shape=(1, 8, 8, 384), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 24, 24, 384

FailedPreconditionError: Attempting to use uninitialized value conv1/W
	 [[Node: conv1/W/read = Identity[T=DT_FLOAT, _class=["loc:@conv1/W"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1/W)]]
	 [[Node: conv1/W/read/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3_conv1/W/read", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv1/W/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.s